Capstone: Personal FAQ Bot
- Unlike generic ChatGPT, this bot only answers from your documents. No hallucination about your experience.
- Retrieval-Augmented Generation: search your docs, find relevant chunks, generate accurate answers.
- Deploy with Gradio for a link you can send to recruiters, clients, or colleagues.
- This pattern scales to company knowledge bases, customer support, internal wikis.
- Documents → Chunk → Embed → Store → Query → Retrieve → Generate → Answer

Libraries & Dependencies: 
- pip install langchain langchain-openai chromadb gradio python-dotenv pypdf
- OPENAI_API_KEY in .env file. Or use GROQ_API_KEY for free alternative.
- Gather your resume, portfolio descriptions, project summaries as PDF or TXT files.
- Single file works: faq_bot.py with docs/ folder for your documents.
- langchain + chromadb + gradio + pypdf



- PyPDFLoader reads PDFs page by page. Works with resumes, reports, portfolios.
- TextLoader for .txt and .md files. Good for project descriptions.
- DirectoryLoader scans a folder and loads all matching files automatically.
- Each document carries metadata (source file, page number) for citations.

In [5]:
from webbrowser import Chrome
from langchain_community.document_loaders import PyPDFLoader
from langchain_classic.chains.query_constructor.base import AttributeInfo
# from langchain_community.document_loaders import DirectoryLoader
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import BedrockEmbeddings, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_classic.chains.query_constructor.base import AttributeInfo
from langchain_openai import OpenAI
from langchain_classic.retrievers import SelfQueryRetriever
from pinecone import Pinecone
from langchain_classic.document_loaders import DirectoryLoader
from langchain_classic.chains import RetrievalQA
import gradio as gr
from langchain_community.vectorstores import Chroma

def chat(message, history):
    return message

llm = OpenAI(temperature=0)
loader = DirectoryLoader("/Users/vkdvamshi/Resumes/", glob="**/*.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents) # Split documents into chunks
vectorstore = Chroma.from_documents(chunks, OpenAIEmbeddings(), persist_directory="./chroma_db") # Create and persist embedding db / vector store on local storage in choma_db
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
RetrievalQA.from_chain_type(llm, retriever=retriever, return_source_documents=True)
gr.ChatInterface(fn=chat, title="Ask About [Your Name]").launch(share=True)



RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}